In [41]:
from datasets import load_dataset
import pandas as pd
# Load the dataset from Hugging Face Hub
dataset = load_dataset('lukebarousse/data_jobs')

# Convert the 'train' split of the dataset into a pandas DataFrame
df = dataset['train'].to_pandas()

In [42]:
# Inspect the first rows
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


In [43]:
print(df.info())  # Summary of columns, non-null counts, and types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785741 entries, 0 to 785740
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   job_title_short        785741 non-null  object 
 1   job_title              785740 non-null  object 
 2   job_location           784696 non-null  object 
 3   job_via                785733 non-null  object 
 4   job_schedule_type      773074 non-null  object 
 5   job_work_from_home     785741 non-null  bool   
 6   search_location        785741 non-null  object 
 7   job_posted_date        785741 non-null  object 
 8   job_no_degree_mention  785741 non-null  bool   
 9   job_health_insurance   785741 non-null  bool   
 10  job_country            785692 non-null  object 
 11  salary_rate            33067 non-null   object 
 12  salary_year_avg        22003 non-null   float64
 13  salary_hour_avg        10662 non-null   float64
 14  company_name           785723 non-nu

In [44]:
# Converting the data column to date time  
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
print(df.dtypes) #inspect 

job_title_short                  object
job_title                        object
job_location                     object
job_via                          object
job_schedule_type                object
job_work_from_home                 bool
search_location                  object
job_posted_date          datetime64[ns]
job_no_degree_mention              bool
job_health_insurance               bool
job_country                      object
salary_rate                      object
salary_year_avg                 float64
salary_hour_avg                 float64
company_name                     object
job_skills                       object
job_type_skills                  object
dtype: object


In [45]:
df.to_pickle('df-pkl')

In [46]:
# Fill Nan Values with median
df_filled = df.copy()
df_filled['salary_year_avg'] = df_filled['salary_year_avg'].fillna(df['salary_year_avg'].median())
df_filled['salary_hour_avg'] = df_filled['salary_hour_avg'].fillna(df['salary_hour_avg'].median())

In [47]:
# Remove duplicates 
df_unique = df_filled.drop_duplicates()

print('Lengh of original df:      ', len(df_filled))
print('Lenth of drop duplicates df:', len(df_unique))
print('Rows Dropped:', len(df_filled)-len(df_unique))


Lengh of original df:       785741
Lenth of drop duplicates df: 785640
Rows Dropped: 101


In [48]:
df_unique = df_unique.drop_duplicates(subset=('job_title', 'company_name'))
print('Lengh of original df:      ', len(df_filled))
print('Lenth of drop duplicates df:', len(df_unique))
print('Rows Dropped:', len(df_filled)-len(df_unique))

Lengh of original df:       785741
Lenth of drop duplicates df: 508042
Rows Dropped: 277699


In [49]:
# Drop the 'salary_rate' column 
df_unique = df_unique.drop(columns=['salary_rate'])

In [50]:
df_data = df_unique.dropna(subset=['job_skills'])

In [51]:
# Convert skills columns into lists
import ast

df_data['job_skills'] = df_data['job_skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_data['job_type_skills'] = df_data['job_type_skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

/var/folders/rw/s684x_bj2ld32gqpw2zyxm740000gn/T/ipykernel_83890/3498588910.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['job_skills'] = df_data['job_skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
/var/folders/rw/s684x_bj2ld32gqpw2zyxm740000gn/T/ipykernel_83890/3498588910.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['job_type_skills'] = df_data['job_type_skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [52]:
# Inspect 
df_data.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,115000.0,45.98,Hewlett Packard Enterprise,"[r, python, sql, nosql, power bi, tableau]","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,115000.0,45.98,ALPHA Augmented Services,"[python, sql, c#, azure, airflow, dax, docker,...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,115000.0,45.98,Southwest Research Institute,"[python, c++, java, matlab, aws, tensorflow, k...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,115000.0,45.98,Kristina Daniel,"[bash, python, oracle, aws, ansible, puppet, j...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."
5,Data Engineer,GCP Data Engineer,Anywhere,via ZipRecruiter,Contractor and Temp work,True,Georgia,2023-11-07 14:01:59,False,False,United States,115000.0,45.98,smart folks inc,"[python, sql, gcp]","{'cloud': ['gcp'], 'programming': ['python', '..."


In [53]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424837 entries, 1 to 785740
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   job_title_short        424837 non-null  object        
 1   job_title              424837 non-null  object        
 2   job_location           424265 non-null  object        
 3   job_via                424837 non-null  object        
 4   job_schedule_type      417469 non-null  object        
 5   job_work_from_home     424837 non-null  bool          
 6   search_location        424837 non-null  object        
 7   job_posted_date        424837 non-null  datetime64[ns]
 8   job_no_degree_mention  424837 non-null  bool          
 9   job_health_insurance   424837 non-null  bool          
 10  job_country            424800 non-null  object        
 11  salary_year_avg        424837 non-null  float64       
 12  salary_hour_avg        424837 non-null  float64  

In [54]:
df_data.to_pickle('df_data-pkl')